In [1]:
from pymavlink import mavutil
import numpy
# Create a connection to the UDP endpoint
master = mavutil.mavlink_connection('udpin:127.0.0.1:14445')
master.wait_heartbeat(timeout=10)
print("connection established")

connection established


In [8]:
def handle_mavlink_message(master):
    """
    Handle a MAVLink message.
    :param master: A MAVLink connection instance.
    """
    # Wait for a new message.
    msg = master.recv_match(blocking=True)

    # If a message has been received, handle it.
    if msg is not None:
        # Handle different message types.
        if msg.get_type() == 'ATTITUDE':
            print(f"Attitude: Roll={msg.roll}, Pitch={msg.pitch}, Yaw={msg.yaw}")
        elif msg.get_type() == 'GPS_RAW_INT':
            print(f"GPS: Lat={msg.lat / 1e7}, Lon={msg.lon / 1e7}, Alt={msg.alt / 1e3}m")
        elif msg.get_type() == 'SYS_STATUS':
            print(f"Battery: Voltage={msg.voltage_battery / 1000.0}V, Current={msg.current_battery / 100.0}A")


In [3]:
while True:
    handle_mavlink_message(master)

GPS: Lat=0.0, Lon=0.0, Alt=-17.0m
Attitude: Roll=0.02476371079683304, Pitch=-0.008160917088389397, Yaw=0.8670065999031067
Attitude: Roll=0.024732952937483788, Pitch=-0.00816312525421381, Yaw=0.8670681715011597
Attitude: Roll=0.024276912212371826, Pitch=-0.008329351432621479, Yaw=0.8673577904701233
Attitude: Roll=0.024450011551380157, Pitch=-0.008275427855551243, Yaw=0.8674519062042236
Attitude: Roll=0.024409307166934013, Pitch=-0.00819904450327158, Yaw=0.8674409985542297
Battery: Voltage=0.0V, Current=-0.01A
GPS: Lat=0.0, Lon=0.0, Alt=-17.0m
Attitude: Roll=0.02445426769554615, Pitch=-0.008231664076447487, Yaw=0.8672835230827332
Attitude: Roll=0.024437211453914642, Pitch=-0.008299263194203377, Yaw=0.8672258853912354
Attitude: Roll=0.024431858211755753, Pitch=-0.008355976082384586, Yaw=0.8670947551727295
Attitude: Roll=0.024404434487223625, Pitch=-0.008354321122169495, Yaw=0.8669286966323853
Attitude: Roll=0.02437613159418106, Pitch=-0.008342179469764233, Yaw=0.866722047328949
Battery: V

KeyboardInterrupt: 

In [1]:
from pymavlink import mavutil
from pymavlink.dialects.v20 import common as mavlink2

# Replace with your connection string
master = mavutil.mavlink_connection("COM3",baud=57600)

# Wait for the heartbeat message to ensure the connection is established
master.wait_heartbeat()

# Clear any existing missions
master.mav.mission_clear_all_send(master.target_system, master.target_component)

# Define the mission items
mission_items = [
    mavlink2.MAVLink_mission_item_message(
        0,  # target_system
        0,  # target_component
        0,  # seq
        mavlink2.MAV_FRAME_GLOBAL_RELATIVE_ALT,  # frame
        mavlink2.MAV_CMD_NAV_WAYPOINT,  # command
        0,  # current
        1,  # autocontinue
        0, 0, 0, 0,  # params 1-4
        34.052235,  # x (latitude)
        -118.243683,  # y (longitude)
        10  # z (altitude)
    )
    # Add more mission items as needed
]

# Send the mission count
master.mav.mission_count_send(master.target_system, master.target_component, len(mission_items))

# Send the mission items
for i, item in enumerate(mission_items):
    master.mav.send(item)
    # Wait for the vehicle to acknowledge the mission item
    master.recv_match(type='MISSION_ACK', blocking=True)

# Set the first mission item as the active mission item
master.mav.mission_set_current_send(master.target_system, master.target_component, 0)


master.mav.command_long_send(
    master.target_system,
    master.target_component,
    mavutil.mavlink.MAV_CMD_COMPONENT_ARM_DISARM,
    0,
    1, 0, 0, 0, 0, 0, 0)